# In this course, you will learn about:

- sqlite - interacting with SQLite databases;
- xml - creating and processing XML files;
- csv - CSV file reading and writing;
- logging - basics logging facility for Python;
- configparser - configuration file parser.

## 1. Sqlite

import sqlite3
conn1 = sqlite3.connect('hello.db')
conn2 = sqlite3.connect(':memory:')


In [21]:
import sqlite3

conn = sqlite3.connect('todo.db')
c = conn.cursor()
c.execute('''CREATE TABLE IF NOT EXISTS tasks (
id INTEGER PRIMARY KEY,
name TEXT NOT NULL,
priority INTEGER NOT NULL
);''')
c.execute('INSERT INTO tasks (name, priority) VALUES (?,?)', ('My first task', 1))
conn.commit()
conn.close()

'''The mysterious characters ? used in the INSERT INTO statement are query parameters that are replaced with the correct values during the execution of the statement. In the above example, the first character ? will be replaced with My first task, while the second will be replaced with l.
This is to avoid an SQL injection attack in which malicious SQL is appended to a query that could possibly destroy our database. You can find more information about SQL injection and possible safeguard measures on the Internet.'''


OperationalError: database is locked

NOTE: Running the above program twice will throw an exception with the following message: sqlite3.OperationError: table tasks already exists. This is because the statement is trying to re-create a table with the same name. The solution to this problem is to modify the query as follows:

CREATE TABLE IF NOT EXISTS tasks (
id INTEGER PRIMARY KEY,
name TEXT NOT NULL,
priority INTEGER NOT NULL
);

In [30]:
import sqlite3

conn = sqlite3.connect('to_do.db')
c = conn.cursor()
c.execute('''CREATE TABLE IF NOT EXISTS tasks (
id INTEGER PRIMARY KEY,
name TEXT NOT NULL,
priority INTEGER NOT NULL
);''')
tasks = [
    ('My first task', 1),
    ('My second task', 5),
    ('My third task', 10),
]
c.executemany('INSERT INTO tasks (name, priority) VALUES (?,?)', tasks)
conn.commit()
conn.close()


In [32]:
# Application refactoring

import sqlite3

class Todo:
    def __init__(self):
        self.conn = sqlite3.connect('to_do.db')
        self.c = self.conn.cursor()
        self.create_task_table()
        
    def create_task_table(self):
        self.c.execute('''CREATE TABLE IF NOT EXISTS tasks (
                     id INTEGER PRIMARY KEY,
                     name TEXT NOT NULL,
                     priority INTEGER NOT NULL
                     );''')
    
    def add_task(self):
        name = input('Enter task name: ')
        priority = int(input('Enter priority: '))
        
        self.c.execute('INSERT INTO tasks (name, priority) VALUES (?,?)', (name, priority))
        self.conn.commit()

app = Todo()
app.add_task()


Enter task name: coding
Enter priority: 9


In [62]:
# Cursor object as an iterator
import sqlite3

conn = sqlite3.connect('to_do.db')
c = conn.cursor()
#all_name = c.execute('SELECT name FROM tasks')
#for r in all_name:
#    print(r)
for row in c.execute('SELECT * FROM tasks'):
    print(row)
conn.close()


('My first task',)
('My second task',)
('My third task',)
('coding',)
(1, 'My first task', 1)
(2, 'My second task', 5)
(3, 'My third task', 10)
(4, 'coding', 9)


In [64]:
# fetchall is good for small amounts of data

import sqlite3

conn = sqlite3.connect('to_do.db')
c = conn.cursor()
c.execute('SELECT * FROM tasks')
rows = c.fetchall()
for row in rows:
    print(row)
conn.close()


(1, 'My first task', 1)
(2, 'My second task', 5)
(3, 'My third task', 10)
(4, 'coding', 9)


In [36]:
# fetchone 
import sqlite3

conn = sqlite3.connect('to_do.db')
c = conn.cursor()
c.execute('SELECT * FROM tasks')
row = c.fetchone()
print(row)
row = c.fetchone()
print(row)
conn.close()


(1, 'My first task', 1)
(2, 'My second task', 5)


In [65]:
# https://edube.org/learn/pcpp1-5/lab-sqlite3-lab-1

import sqlite3

class Todo:
    def __init__(self):
        self.conn = sqlite3.connect('my_todo.db')
        self.c = self.conn.cursor()
        self.create_task_table()
        
    def create_task_table(self):
        self.c.execute('''CREATE TABLE IF NOT EXISTS tasks (
                     id INTEGER PRIMARY KEY,
                     name TEXT NOT NULL,
                     priority INTEGER NOT NULL
                     );''')
 
    def find_task(self, task_name):                 
        
        for row in self.c.execute('SELECT name FROM tasks'):
            print(row)
            if task_name in row:
                return True
        return False
    
    def add_task(self):
        
        name = input('Enter task name(Enter to exit...): ')
        while name:
            
            priority = int(input('Enter priority (> 1): '))
            while priority <=1:
                priority = int(input('The priority must > 1, Enter again: '))
            if self.find_task(name):
                print("You already have this task in your list!")
            else:
                self.c.execute('INSERT INTO tasks (name, priority) VALUES (?,?)', (name, priority))
                self.conn.commit()
            name = input('Enter task name(Enter to exit...): ')
            

        
        
    def show_tasks(self):
        pass
        for row in self.c.execute('SELECT * FROM tasks'):
            print(row)
    
app = Todo()
app.add_task()
app.show_tasks()



Enter task name(Enter to exit...): coding
Enter priority (> 1): 3
('coding',)
You already have this task in your list!
Enter task name(Enter to exit...): 
(1, 'coding', 9)
(2, 'playing', 1)
(3, 'fishing', 2)
(4, 'gaming', 3)
(5, 'eating', 4)
(6, 'sleeping', 6)
(7, 'running', 7)


In [69]:
#update
import sqlite3

conn = sqlite3.connect('todo.db')
c = conn.cursor()
c.execute('UPDATE tasks SET priority = ? WHERE id = ?', (20, 1))
c.commit()
c.close()


OperationalError: database is locked

In [68]:
#delete
import sqlite3

conn = sqlite3.connect('todo.db')
c = conn.cursor()
c.execute('DELETE FROM tasks WHERE id = ?', (1,))
c.commit()
c.close()


In [75]:
# https://edube.org/learn/pcpp1-5/lab-sqlite3-lab-2

import sqlite3

class Todo:
    def __init__(self):
        self.conn = sqlite3.connect('my_todo.db')
        self.c = self.conn.cursor()
        self.create_task_table()
        
    def create_task_table(self):
        self.c.execute('''CREATE TABLE IF NOT EXISTS tasks (
                     id INTEGER PRIMARY KEY,
                     name TEXT NOT NULL,
                     priority INTEGER NOT NULL
                     );''')
 
    def find_task(self, task_name):                         
        for row in self.c.execute('SELECT name FROM tasks'):
            print(row)
            if task_name in row:
                return True
        return False
    
    def add_task(self):        
        name = input('Enter task name(Enter to exit...): ')
        while name:           
            priority = int(input('Enter priority (> 1): '))
            while priority <=1:
                priority = int(input('The priority must > 1, Enter again: '))
            if self.find_task(name):
                print("You already have this task in your list!")
            else:
                self.c.execute('INSERT INTO tasks (name, priority) VALUES (?,?)', (name, priority))
                self.conn.commit()
            name = input('Enter task name(Enter to exit...): ')        
        
    def show_tasks(self):
        pass
        for row in self.c.execute('SELECT * FROM tasks'):
            print(row)
    
    def delete_task(self):
        pass
        task_id = int(input('Enter task id (> 1): '))
        self.c.execute('DELETE FROM tasks WHERE id = ?', (task_id,))
        self.c.commit()
        
    def change_priority(self):
        pass
        task_id = int(input('Enter task id (> 1): '))

        priority = int(input('Enter priority (> 1): '))
        while priority <=1:
            priority = int(input('The priority must > 1, Enter again: '))
        self.c.execute('UPDATE tasks SET priority = ? WHERE id = ?', (priority, task_id))
        self.c.commit()
    
    


app = Todo()
app.show_tasks()
#app.add_task()
app.delete_task()
app.change_priority()
app.c.close()

(1, 'coding', 9)
(2, 'playing', 1)
(3, 'fishing', 2)
(4, 'gaming', 3)
(5, 'eating', 4)
(6, 'sleeping', 6)
(7, 'running', 7)
(8, 'watching', 5)
Enter task id (> 1): 8


OperationalError: database is locked

## 2. XML processing in Python
- xml.etree.ElementTree – has a very simple API for analyzing and creating XML data. It's an excellent choice for people who have never worked with the Document Object Model (DOM) before.
- xml.dom.minidom – is the minimum implementation of the Document Object Model (DOM). Using the DOM, the approach to an XML document is slightly different, because it's parsed into a tree structure in which each node is an object.
- xml.sax – SAX is an acronym for “Simple API for XML”. SAX is an interface in Python for event-driven XML document analysis. Unlike the above modules, analyzing a simple XML document using this module requires more work.
</br></br>
- prolog – the first (optional) line of the document. In the prolog, you can specify character encoding, e.g., <?xml version="1.0" encoding="ISO-8859-2"?> changes the default character encoding (UTF-8) to ISO-8859-2.
- root element – the XML document must have one root element that contains all other elements. In the example below, the main element is the data tag.
- elements – these consist of opening and closing tags. The elements include text, attributes, and other child elements. In the example below, we can find the book element with the title attribute and two child elements (author and year).
- attributes – these are placed in the opening tags. They consist of key-value pairs, e.g., title = "The Little Prince".

In [3]:
import xml.etree.ElementTree as ET

tree = ET.parse('books.xml')
root = tree.getroot()
'''NOTE: The fromstring method doesn't return the ElementTree object, but instead returns the root element represented by the Element class.

'''
import xml.etree.ElementTree as ET

root = ET.fromstring(your_xml_as_string)


NameError: name 'your_xml_as_string' is not defined

In [2]:
import xml.etree.ElementTree as ET

tree = ET.parse('books.xml')
root = tree.getroot()
print('The root tag is:', root.tag)
print('The root has the following children:')
for child in root:
    print(child.tag, child.attrib)


The root tag is: data
The root has the following children:
book {'title': 'The Little Prince'}
book {'title': 'Hamlet'}


In [5]:
import xml.etree.ElementTree as ET

tree = ET.parse('books.xml')
root = tree.getroot()
print("My books:\n")
for book in root:
    print('Title: ', book.attrib['title'])
    print('Author:', book[0].text) # Index 0 refers to the first child 
    print('Year: ', book[1].text, '\n') # Index 1 refers to the second child 
'''NOTE: Indexes are also used in deeper nesting, e.g., root [0] [0] .text returns the first book element, and then displays the text placed in its first child.

'''

My books:

Title:  The Little Prince
Author: Antoine de Saint-Exupéry
Year:  1943 

Title:  Hamlet
Author: William Shakespeare
Year:  1603 



'NOTE: Indexes are also used in deeper nesting, e.g., root [0] [0] .text returns the first book element, and then displays the text placed in its first child.\n\n'

In [6]:
# iter
import xml.etree.ElementTree as ET

tree = ET.parse('books.xml')
root = tree.getroot()
for author in root.iter('author'):
    print(author.text)


Antoine de Saint-Exupéry
William Shakespeare


In [16]:
#findall
import xml.etree.ElementTree as ET

tree = ET.parse('books.xml')
root = tree.getroot()
for author in root.findall('author'):
    print(author.text)
'''the findall method only searches the children at the first nesting level.'''

'the findall method only searches the children at the first nesting level.'

In [17]:
import xml.etree.ElementTree as ET

tree = ET.parse('books.xml')
root = tree.getroot()
for book in root.findall('book'):
    print(book.get('title'))
'''NOTE: The findall method also accepts an XPath expression. We encourage you to deepen your knowledge of XPath expressions and apply it to the example shown.

'''

The Little Prince
Hamlet


In [18]:
# find
import xml.etree.ElementTree as ET

tree = ET.parse('books.xml')
root = tree.getroot()
print(root.find('book').get('title'))


The Little Prince


In [125]:
# https://edube.org/learn/pcpp1-5/lab-xml-lab-1
import xml.etree.ElementTree as ET

class TemperatureConverter:
    def __init__(self, c):
        self.c = c
        
    def convert_celsius_to_fahrenheit(self):
        return round( (9/5 * self.c + 32), 2)
    
class ForecastXmlParser:
    def __init__(self, xml_file):
        self.tree = ET.parse(xml_file)
        self.root = tree.getroot()

    def parse(self):
        for item in self.root:
            t = TemperatureConverter(int(item[1].text))
            f = t.convert_celsius_to_fahrenheit()
            print("{}: {} Celsius, {} Fahrenheit".format(item[0].text, item[1].text,f)) 

forecast = ForecastXmlParser("forecast.xml")
forecast.parse()

Monday: 28 Celsius, 82.4 Fahrenheit
Tuesday: 27 Celsius, 80.6 Fahrenheit
Wednesday: 28 Celsius, 82.4 Fahrenheit
Thursday: 29 Celsius, 84.2 Fahrenheit
Friday: 29 Celsius, 84.2 Fahrenheit
Saturday: 31 Celsius, 87.8 Fahrenheit
Sunday: 32 Celsius, 89.6 Fahrenheit


In [127]:
#modify 
import xml.etree.ElementTree as ET

tree = ET.parse('books.xml')
root = tree.getroot()
for child in root:
    child.tag = 'movie'
    print(child.tag, child.attrib)
    for sub_child in child:
        print(sub_child.tag, ':', sub_child.text)


movie {'title': 'The Little Prince'}
author : Antoine de Saint-Exupéry
year : 1943
movie {'title': 'Hamlet'}
author : William Shakespeare
year : 1603


In [128]:
#modify remove

import xml.etree.ElementTree as ET

tree = ET.parse('books.xml')
root = tree.getroot()
for child in root:
    child.tag = 'movie'
    child.remove(child.find('author'))
    child.remove(child.find('year'))
    print(child.tag, child.attrib)
    for sub_child in child:
        print(sub_child.tag, ':', sub_child.text)


movie {'title': 'The Little Prince'}
movie {'title': 'Hamlet'}


In [129]:
#modify set

import xml.etree.ElementTree as ET

tree = ET.parse('books.xml')
root = tree.getroot()
for child in root:
    child.tag = 'movie'
    child.remove(child.find('author'))
    child.remove(child.find('year'))
    child.set('rate', '5')
    print(child.tag, child.attrib)
    for sub_child in child:
        print(sub_child.tag, ':', sub_child.text)


movie {'title': 'The Little Prince', 'rate': '5'}
movie {'title': 'Hamlet', 'rate': '5'}


In [130]:
# write
import xml.etree.ElementTree as ET

tree = ET.parse('books.xml')
root = tree.getroot()
for child in root:
    child.tag = 'movie'
    child.remove(child.find('author'))
    child.remove(child.find('year'))
    child.set('rate', '5')
    print(child.tag, child.attrib)
    for sub_child in child:
        print(sub_child.tag, ':', sub_child.text)

tree.write('movies.xml', 'UTF-8', True)

movie {'title': 'The Little Prince', 'rate': '5'}
movie {'title': 'Hamlet', 'rate': '5'}


In [131]:
# Building an XML document (part 1)
import xml.etree.ElementTree as ET

root = ET.Element('data')
ET.dump(root)


<data />


In [132]:
# Building an XML document (part 2)

import xml.etree.ElementTree as ET

root = ET.Element('data')
movie_1 = ET.SubElement(root, 'movie', {'title': 'The Little Prince', 'rate': '5'})
movie_2 = ET.SubElement(root, 'movie', {'title': 'Hamlet', 'rate': '5'})
ET.dump(root)


<data><movie title="The Little Prince" rate="5" /><movie title="Hamlet" rate="5" /></data>


In [158]:
# https://edube.org/learn/pcpp1-5/lab-xml-lab-2
import xml.etree.ElementTree as ET

root = ET.Element('shop')
vegan = ET.SubElement(root, 'category', {'name': "Vegan Products"})

prod1 = ET.SubElement(vegan, 'product ', {'name': 'Good Morning Sunshine'})
prod11 = ET.SubElement(prod1, 'type')
prod11.text = 'cereals'
prod12 = ET.SubElement(prod1, 'producer')
prod12.text = 'OpenEDG Testing Service'
prod13 = ET.SubElement(prod1, 'price')
prod13.text = '9.90'
prod14 = ET.SubElement(prod1, 'curreency')
prod14.text = 'USD'

prod2 = ET.SubElement(vegan, 'product ', {'name': 'Spaghetti Veganietto'})
prod21 = ET.SubElement(prod2, 'type')
prod21.text = 'pasta'
prod22 = ET.SubElement(prod1, 'producer')
prod22.text = 'Programmers Eat Pasta'
prod23 = ET.SubElement(prod1, 'price')
prod23.text = '15.49'
prod24 = ET.SubElement(prod1, 'curreency')
prod24.text = 'EUR'

prod3 = ET.SubElement(vegan, 'product ', {'name': 'Fantastic Almond Milk'})
prod31 = ET.SubElement(prod1, 'type')
prod31.text = 'beverages'
prod32 = ET.SubElement(prod1, 'producer')
prod32.text = 'Drinks4Coders Inc.'
prod33 = ET.SubElement(prod1, 'price')
prod33.text = '19.75'
prod34 = ET.SubElement(prod1, 'curreency')
prod34.text = 'USD'


ET.dump(root)


tree = ET.ElementTree(root)
tree.write('shop.xml', 'UTF-8', True)


<shop><category name="Vegan Products"><product  name="Good Morning Sunshine"><type>cereals</type><producer>OpenEDG Testing Service</producer><price>9.90</price><curreency>USD</curreency><producer>Programmers Eat Pasta</producer><price>15.49</price><curreency>EUR</curreency><type>beverages</type><producer>Drinks4Coders Inc.</producer><price>19.75</price><curreency>USD</curreency></product ><product  name="Spaghetti Veganietto"><type>pasta</type></product ><product  name="Fantastic Almond Milk" /></category></shop>


## The CSV module in Python


In [159]:
import csv

with open('contacts.csv', newline='') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    for row in reader:
        print(row)

['Name', 'Phone']
['mother', '222-555-101']
['father', '222-555-102']
['wife', '222-555-103']
['mother-in-law', '222-555-104']


In [161]:
import csv

with open('contacts.csv', newline='') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    for row in reader:
        print(','.join(row))
'''NOTE: The newline='' argument added to the open function protects us from incorrect interpretation of the newline character on different platforms.

'''

Name,Phone
mother,222-555-101
father,222-555-102
wife,222-555-103
mother-in-law,222-555-104


"NOTE: The newline='' argument added to the open function protects us from incorrect interpretation of the newline character on different platforms.\n\n"

In [162]:
import csv

with open('contacts.csv', newline='') as csvfile:
    fieldnames = ['Name', 'Phone']
    reader = csv.DictReader(csvfile, fieldnames=fieldnames)
    for row in reader:
        print(row['Name'], row['Phone'])

Name Phone
mother 222-555-101
father 222-555-102
wife 222-555-103
mother-in-law 222-555-104


In [191]:
#https://edube.org/learn/pcpp1-5/lab-csv-lab-1
import csv

class PhoneContact:
    def __init__(self, name, phone):
        self.name = name
        self.phone = phone
    


class Phone:
    def __init__(self, csv_file):
        
        self.csv_file = csv_file
        self.contacts = []
        self.load_contacts_from_csv()
        
    def load_contacts_from_csv(self):
        with open(self.csv_file, newline='') as csvfile:
            fieldnames = ['Name', 'Phone']
            reader = csv.DictReader(csvfile, fieldnames=fieldnames)
            self.contacts = [PhoneContact(row['Name'], row['Phone']) for row in reader ]
                
    def search_contacts(self):
        found = False
        search_contact = input("Search contacts:")
        for c in self.contacts:
            if search_contact in c.name or search_contact in c.phone:
                print("{} ({})".format(c.name,c.phone))
                found = True
        if not found:
            print("No contacts found")
            
phone = Phone('contacts.csv')  
# phone.load_contacts_from_csv()
for c in phone.contacts:
    print(c.name,c.phone)
print()
phone.search_contacts()

Name Phone
mother 222-555-101
father 222-555-102
wife 222-555-103
mother-in-law 222-555-104

Search contacts:1234
No contacts found


In [192]:
# save
import csv

with open('exported_contacts.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile, delimiter=',')
    
    writer.writerow(['Name', 'Phone'])
    writer.writerow(['mother', '222-555-101'])
    writer.writerow(['father', '222-555-102'])
    writer.writerow(['wife', '222-555-103'])
    writer.writerow(['mother-in-law', '222-555-104'])

In [193]:
import csv

with open('exported_contacts.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    
    writer.writerow(['Name', 'Phone'])
    writer.writerow(['mother', '222-555-101'])
    writer.writerow(['father', '222-555-102'])
    writer.writerow(['wife', '222-555-103'])
    writer.writerow(['mother-in-law', '222-555-104'])
    writer.writerow(['grandmother, grandfather', '222-555-105'])
'''Below are other constants that we can use as the value of the quoting argument:

csv.QUOTE_ALL – quotes all values

csv.QUOTE_NONNUMERIC – quotes only non-numeric values

csv.QUOTE_NONE – doesn't quote any values. It's not a good idea to set this value if you have special characters that require quoting, because this will raise an error

NOTE: The quotechar and quoting parameters can also be used in the reader function. See the documentation for more information.'''

"Below are other constants that we can use as the value of the quoting argument:\n\ncsv.QUOTE_ALL – quotes all values\n\ncsv.QUOTE_NONNUMERIC – quotes only non-numeric values\n\ncsv.QUOTE_NONE – doesn't quote any values. It's not a good idea to set this value if you have special characters that require quoting, because this will raise an error\n\nNOTE: The quotechar and quoting parameters can also be used in the reader function. See the documentation for more information."

In [194]:
import csv

with open('exported_contacts.csv', 'w', newline='') as csvfile:
    fieldnames = ['Name', 'Phone']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    
    writer.writeheader()
    writer.writerow({'Name': 'mother', 'Phone': '222-555-101'})
    writer.writerow({'Name': 'father', 'Phone': '222-555-102'
    writer.writerow({'Name': 'wife', 'Phone': '222-555-103'})
    writer.writerow({'Name': 'mother-in-law', 'Phone': '222-555-104'})
    writer.writerow({'Name': 'grandmother, grandfather and auntie', 'Phone': '222-555-105'})

In [29]:
#https://edube.org/learn/pcpp1-5/lab-csv-lab-2
#using the writer function or the DictWriter class.    
#Exam Name,Number of Candidates,Number of Passed Exams,Number of Failed Exams,Best Score,Worst Score
#Maths,8,4,6,90,33
#Physics,3,0,3,66,50
#Biology,5,2,3,88,23    
import csv
results = [['Maths',0,0,0,0,100],['Physics',0,0,0,0,100],['Biology',0,0,0,0,100]]
with open('exam_results.csv', newline='') as csvfile:
    #fieldnames = ['Exam Name', 'Candidate ID', 'Score', 'Grade']
    reader = csv.reader(csvfile, delimiter=',')
    for row in reader:
#         print(row)
        if row[0]=='Maths':
            results[0][1] += 1
#             print(row[3])
            if row[3] == 'Fail':
                 results[0][3] += 1
            else:
                results[0][2]  += 1
            if int(row[2])> results[0][4]:
                results[0][4]=int(row[2])
            if int(row[2])< results[0][5]:
                results[0][5]=int(row[2])
                
        if row[0]=='Physics':
            pass
            results[1][1] += 1
#             print(row[3])
            if row[3] == 'Fail':
                 results[1][3] += 1
            else:
                results[1][2]  += 1
            if int(row[2])> results[1][4]:
                results[1][4]=int(row[2])
            if int(row[2])< results[1][5]:
                results[1][5]=int(row[2])
                
        if row[0]=='Biology':
            pass
            results[2][1] += 1
#             print(row[3])
            if row[3] == 'Fail':
                 results[2][3] += 1
            else:
                results[2][2]  += 1
            if int(row[2])> results[2][4]:
                results[2][4]=int(row[2])
            if int(row[2])< results[0][5]:
                results[2][5]=int(row[2])
                
print(results)
with open('results.csv', 'w', newline='') as csvfile:
    fieldnames = ['Exam Name','Number of Candidates','Number of Passed Exams','Number of Failed Exams','Best Score','Worst Score']
    writer = csv.writer(csvfile, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
#     writer.writeheader()
    for row in results:
        
        writer.writerow(row)

[['Maths', 10, 4, 6, 90, 33], ['Physics', 3, 0, 3, 66, 50], ['Biology', 5, 2, 3, 88, 23]]


## Logging in Python

import logging
Level name	Value
CRITICAL	50
ERROR	40
WARNING	30
INFO	20
DEBUG	10
NOTSET	0



In [1]:
import logging

logger = logging.getLogger()
hello_logger = logging.getLogger('hello')
hello_world_logger = logging.getLogger('hello.world')
recommended_logger = logging.getLogger(__name__)

In [31]:
import logging

logging.basicConfig()

logger = logging.getLogger()

logger.critical('Your CRITICAL message')
logger.error('Your ERROR message')
logger.warning('Your WARNING message')
logger.info('Your INFO message')
logger.debug('Your DEBUG message')

CRITICAL:root:Your CRITICAL message
ERROR:root:Your ERROR message


In [32]:
import logging

logging.basicConfig()

logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

logger.critical('Your CRITICAL message')
logger.error('Your ERROR message')
logger.warning('Your WARNING message')
logger.info('Your INFO message')
logger.debug('Your DEBUG message')

CRITICAL:root:Your CRITICAL message
ERROR:root:Your ERROR message
INFO:root:Your INFO message
DEBUG:root:Your DEBUG message


In [3]:
# Basic configuration (part 1)

import logging

logging.basicConfig(level=logging.CRITICAL, filename='prod.log', filemode='a')

logger = logging.getLogger()

logger.critical('Your CRITICAL message')
logger.error('Your ERROR message')
logger.warning('Your WARNING message')
logger.info('Your INFO message')
logger.debug('Your DEBUG message')

In [9]:
#Basic configuration (part 2)

import logging

FORMAT = '%(name)s:%(levelname)s:%(asctime)s:%(message)s'

logging.basicConfig(level=logging.CRITICAL, filename='prod.log', filemode='a', format=FORMAT)

logger = logging.getLogger()

logger.critical('Your CRITICAL message')
logger.error('Your ERROR message')
logger.warning('Your WARNING message')
logger.info('Your INFO message')
logger.debug('Your DEBUG message')

#doesn't work here
'''root:CRITICAL:2022-02-17 09:09:28,740:Your CRITICAL message
'''

'root:CRITICAL:2022-02-17 09:09:28,740:Your CRITICAL message\n'

In [11]:
# first handler
import logging

logger = logging.getLogger(__name__)

handler = logging.FileHandler('prod.log', mode='w')
handler.setLevel(logging.CRITICAL)

logger.addHandler(handler)

logger.critical('Your CRITICAL message')
logger.error('Your ERROR message')
logger.warning('Your WARNING message')
logger.info('Your INFO message')
logger.debug('Your DEBUG message')

'''NOTE: Each logger can have several handlers added. One handler can save logs to a file, while another can send them to an external service. In order to process messages with a level lower than WARNING by added handlers, it's necessary to set this level threshold in the root logger.

'''

"NOTE: Each logger can have several handlers added. One handler can save logs to a file, while another can send them to an external service. In order to process messages with a level lower than WARNING by added handlers, it's necessary to set this level threshold in the root logger.\n\n"

In [12]:
# first formatter
import logging

FORMAT = '%(name)s:%(levelname)s:%(asctime)s:%(message)s'

logger = logging.getLogger(__name__)

handler = logging.FileHandler('prod.log', mode='w')
handler.setLevel(logging.CRITICAL)

formatter = logging.Formatter(FORMAT)
handler.setFormatter(formatter)

logger.addHandler(handler)

logger.critical('Your CRITICAL message')
logger.error('Your ERROR message')
logger.warning('Your WARNING message')
logger.info('Your INFO message')
logger.debug('Your DEBUG message')

In [24]:
#https://edube.org/learn/pcpp1-5/logging-lab-01
import logging
import random
import time

FORMAT = '%(levelname)s-TEMPERATURE_IN_CELSIUS UNIT=>%(levelname)s - %(temp) C'
logger = logging.getLogger(__name__)
handler = logging.FileHandler(' battery_temperature.log', mode='w')
handler.setLevel(logging.DEBUG)

formatter = logging.Formatter(FORMAT)
handler.setFormatter(formatter)

logger.addHandler(handler)
i = 0
while i<60:
    temp = random.randint(20, 40)
    if temp>35:
        print(temp)
        logger.critical('TEMPERATURE_IN_CELSIUS > 35')
#     logger.error('Your ERROR message')
    if 30<=temp<=35:
        logger.warning('TEMPERATURE_IN_CELSIUS >= 30 AND TEMPERATURE_IN_CELSIUS <= 35')
#     logger.info('Your INFO message')
    if temp<20:
        logger.debug('TEMPERATURE_IN_CELSIUS < 20')
    time.sleep(1)
    i += 1
    print(i)

1
36


--- Logging error ---
Traceback (most recent call last):
  File "C:\Anaconda3\lib\logging\__init__.py", line 434, in format
    return self._format(record)
  File "C:\Anaconda3\lib\logging\__init__.py", line 430, in _format
    return self._fmt % record.__dict__
KeyError: 'temp'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Anaconda3\lib\logging\__init__.py", line 1083, in emit
    msg = self.format(record)
  File "C:\Anaconda3\lib\logging\__init__.py", line 927, in format
    return fmt.format(record)
  File "C:\Anaconda3\lib\logging\__init__.py", line 666, in format
    s = self.formatMessage(record)
  File "C:\Anaconda3\lib\logging\__init__.py", line 635, in formatMessage
    return self._style.format(record)
  File "C:\Anaconda3\lib\logging\__init__.py", line 436, in format
    raise ValueError('Formatting field not found in record: %s' % e)
ValueError: Formatting field not found in record: 'temp'
Call stack:
  F

2
39


--- Logging error ---
Traceback (most recent call last):
  File "C:\Anaconda3\lib\logging\__init__.py", line 434, in format
    return self._format(record)
  File "C:\Anaconda3\lib\logging\__init__.py", line 430, in _format
    return self._fmt % record.__dict__
KeyError: 'temp'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Anaconda3\lib\logging\__init__.py", line 1083, in emit
    msg = self.format(record)
  File "C:\Anaconda3\lib\logging\__init__.py", line 927, in format
    return fmt.format(record)
  File "C:\Anaconda3\lib\logging\__init__.py", line 666, in format
    s = self.formatMessage(record)
  File "C:\Anaconda3\lib\logging\__init__.py", line 635, in formatMessage
    return self._style.format(record)
  File "C:\Anaconda3\lib\logging\__init__.py", line 436, in format
    raise ValueError('Formatting field not found in record: %s' % e)
ValueError: Formatting field not found in record: 'temp'
Call stack:
  F

3
4
5
6
7
8
9
10
37


...

55
37


--- Logging error ---
Traceback (most recent call last):
  File "C:\Anaconda3\lib\logging\__init__.py", line 434, in format
    return self._format(record)
  File "C:\Anaconda3\lib\logging\__init__.py", line 430, in _format
    return self._fmt % record.__dict__
KeyError: 'temp'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Anaconda3\lib\logging\__init__.py", line 1083, in emit
    msg = self.format(record)
  File "C:\Anaconda3\lib\logging\__init__.py", line 927, in format
    return fmt.format(record)
  File "C:\Anaconda3\lib\logging\__init__.py", line 666, in format
    s = self.formatMessage(record)
  File "C:\Anaconda3\lib\logging\__init__.py", line 635, in formatMessage
    return self._style.format(record)
  File "C:\Anaconda3\lib\logging\__init__.py", line 436, in format
    raise ValueError('Formatting field not found in record: %s' % e)
ValueError: Formatting field not found in record: 'temp'
Call stack:
  F

56
40


--- Logging error ---
Traceback (most recent call last):
  File "C:\Anaconda3\lib\logging\__init__.py", line 434, in format
    return self._format(record)
  File "C:\Anaconda3\lib\logging\__init__.py", line 430, in _format
    return self._fmt % record.__dict__
KeyError: 'temp'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Anaconda3\lib\logging\__init__.py", line 1083, in emit
    msg = self.format(record)
  File "C:\Anaconda3\lib\logging\__init__.py", line 927, in format
    return fmt.format(record)
  File "C:\Anaconda3\lib\logging\__init__.py", line 666, in format
    s = self.formatMessage(record)
  File "C:\Anaconda3\lib\logging\__init__.py", line 635, in formatMessage
    return self._style.format(record)
  File "C:\Anaconda3\lib\logging\__init__.py", line 436, in format
    raise ValueError('Formatting field not found in record: %s' % e)
ValueError: Formatting field not found in record: 'temp'
Call stack:
  F

57
58
59
60


## configparser 

Currently, many popular services provide an API that we can use in our applications. Integration with these services requires authentication using data such as a login and password, or simply an access token.A better solution is to use the configuration file, which will be read by the code. In Python, this is possible thanks to a module called configparser.</br>
config.ini</br>
[DEFAULT]</br>
host = localhost # This is a comment.</br></br>
[mariadb]</br>
name = hello</br>
user = user</br>
password = password</br>
</br>
[redis]</br>
port = 6379</br>
db = 0



In [26]:
import configparser

config = configparser.ConfigParser()
print(config.read('config.ini'))

print('Sections:', config.sections(),'\n')

print('mariadb section:')
print('Host:', config['mariadb']['host'])
print('Database:', config['mariadb']['name'])
print('Username:', config['mariadb']['user'])
print('Password:', config['mariadb']['password'], '\n')

print('redis section:')
print('Host:', config['redis']['host'])
print('Port:', int(config['redis']['port']))
print('Database number:', int(config['redis']['db']))
# getmethod
print('Host:', config.get('mariadb', 'host')) # print('Host:', config['mariadb']['host'])



['config.ini']
Sections: ['mariadb', 'redis'] 

mariadb section:
Host: localhost # This is a comment.
Database: hello
Username: user
Password: password 

redis section:
Host: localhost # This is a comment.
Port: 6379
Database number: 0
Host: localhost # This is a comment.


In [27]:
#Reading configuration from other sources
# read_dict, read_file, read_string

import configparser

config = configparser.ConfigParser()

dict = {
    'DEFAULT': {
        'host': 'localhost'
    },
    'mariadb': {
        'name': 'hello',
        'user': 'root',
        'password': 'password'
    },
    'redis': {
        'port': 6379,
        'db': 0
    }
}

config.read_dict(dict)

print('Sections:', config.sections(),'\n')

print('mariadb section:')
print('Host:', config['mariadb']['host'])
print('Database:', config['mariadb']['name'])
print('Username:', config['mariadb']['user'])
print('Password:', config['mariadb']['password'], '\n')

print('redis section:')
print('Host:', config['redis']['host'])
print('Port:', int(config['redis']['port']))
print('Database number:', int(config['redis']['db']))


Sections: ['mariadb', 'redis'] 

mariadb section:
Host: localhost
Database: hello
Username: root
Password: password 

redis section:
Host: localhost
Port: 6379
Database number: 0


In [28]:
#creating a configure file
import configparser

config = configparser.ConfigParser()

config['DEFAULT'] = {'host': 'localhost'}
config['mariadb'] = {'name': 'hello',
                     'user': 'root',
                     'password': 'password'}
config['redis'] = {'port': 6379,
                   'db': 0}

with open('config.ini', 'w') as configfile:
    config.write(configfile)

In [ ]:
# Interpolating values

[DEFAULT]
host = localhost

[mariadb]
name = hello
user = user
password = password

[redis]
port = 6379
db = 0
dsn = redis://%(host)s
    
'''The configuration file has been extended with another option called dsn. Its value contains the placeholder %(host)s, which needs to be replaced by an appropriate value.

Placing any key between % and s informs the parser of the need to interpolate. Of course, all the work is done for us, and we only get the ready results.

For the dsn option, it'll be the following string: redis://localhost. Note that the placeholder %(host)s has been replaced by the value stored in the host option.

'''